In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="qwen3:4b-instruct-2507-q4_K_M", temperature=0.1)

In [2]:
from agents.csv_to_graph import CsvToGraphAgent
from agents.csv_to_graph import State
from utils.duckdb_helpers import execute_sql_query
from langchain_core.messages import HumanMessage

agent = CsvToGraphAgent(llm, "sandbox/movies-w-year.csv")

test_question="How does ratings for the romantic movies and action movies compare?"
state = State(
    messages=[
        HumanMessage(role="user", content=test_question)
    ]
)

print(agent.parse_user_question(state))
print(agent.get_unique_nouns(state))
print(agent.get_sql_query(state))
print(agent.validate_and_fix_sql(state))

print(execute_sql_query(state["valid_sql"]))
print(agent.execute_sql(state))

print(agent.choose_visualization(state))

vega_spec = agent.vega_lite_visualizer(state)
print(vega_spec)

vega_spec = agent.llm_json_fix(vega_spec)
print(str(vega_spec))


{'messages': [HumanMessage(content='How does ratings for the romantic movies and action movies compare?', additional_kwargs={}, response_metadata={}, role='user')], 'parsed_question': QueryParsing(is_relevant=True, relevant_tables=[RelevantTable(table_name='MOVIES_WITH_YEAR', columns=['Genre', 'IMDB Rating'], noun_columns=['Genre'])])}
{'unique_nouns': [{'Comedy', 'Documentary', 'Musical', 'Black Comedy', 'Action', 'Horror', 'Romantic Comedy', 'Adventure', 'Western', 'Concert', 'Drama', 'Thriller'}]}
{'messages': [HumanMessage(content='How does ratings for the romantic movies and action movies compare?', additional_kwargs={}, response_metadata={}, role='user')], 'parsed_question': QueryParsing(is_relevant=True, relevant_tables=[RelevantTable(table_name='MOVIES_WITH_YEAR', columns=['Genre', 'IMDB Rating'], noun_columns=['Genre'])]), 'unique_nouns': [{'Action', 'Documentary', 'Musical', 'Comedy', 'Black Comedy', 'Horror', 'Western', 'Adventure', 'Romantic Comedy', 'Drama', 'Concert', 'Th

In [3]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver


# Initialize checkpointer for conversation memory
checkpointer = InMemorySaver()

# Create graph builder
graph_builder = StateGraph(State)


# Add nodes
graph_builder.add_node("parse_user_question", agent.parse_user_question)
graph_builder.add_node("get_unique_nouns", agent.get_unique_nouns)
graph_builder.add_node("get_sql_query", agent.get_sql_query)
graph_builder.add_node("validate_and_fix_sql", agent.validate_and_fix_sql)
graph_builder.add_node("execute_sql", agent.execute_sql)
graph_builder.add_node("choose_visualization", agent.choose_visualization)
graph_builder.add_node("visualize", agent.visualize)
graph_builder.add_node("answer_with_visual", agent.answer_with_visual)
graph_builder.add_node("answer_with_data", agent.answer_with_data)
##graph_builder.add_node("vega_lite_visualizer", vega_lite_visualizer)

graph_builder.add_edge(START, "parse_user_question")
graph_builder.add_conditional_edges("parse_user_question", agent.route_after_question_parse)
graph_builder.add_edge("get_unique_nouns", "get_sql_query")
graph_builder.add_edge("get_sql_query", "validate_and_fix_sql")
graph_builder.add_edge("validate_and_fix_sql", "execute_sql")
graph_builder.add_edge("execute_sql", "choose_visualization")
graph_builder.add_conditional_edges("choose_visualization", agent.route_after_choose_visualization)
graph_builder.add_edge("visualize", "answer_with_visual")

# Compile graph with checkpointer
agent_graph = graph_builder.compile(
    checkpointer=checkpointer,
)

In [ ]:
import uuid
from langchain_core.messages import HumanMessage
import gradio as gr
from PIL import Image

def create_thread_id():
    """Generate a unique thread ID for each conversation"""
    return {"configurable": {"thread_id": str(uuid.uuid4())}}

def clear_session():
    """Clear thread for new conversation"""
    global config
    agent_graph.checkpointer.delete_thread(config["configurable"]["thread_id"])
    config = create_thread_id()

def chat_with_agent(message, history):
    current_state = agent_graph.get_state(config)
    
    if current_state.next:
        # Resume interrupted conversation
        agent_graph.update_state(config,{"messages": [HumanMessage(content=message.strip())]})
        result = agent_graph.invoke(None, config)
    else:
        # Start new query
        result = agent_graph.invoke({"messages": [HumanMessage(content=message.strip())]},config)
    
    image = result.get("generated_chart", None)

    print(result)
    print(image)
    
    if (image is not None):
        # Convert PNG bytes → PIL Image
        pil_image = Image.open(image)

        return [result['messages'][-1].content, gr.Image(pil_image)]

    return result['messages'][-1].content

# Initialize thread configuration
config = create_thread_id()

# Create Gradio interface
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        height=600,
        placeholder="<strong>Ask me anything!</strong><br><em>I'll search, reason, and act to give you the best answer :)</em>"
    )
    chatbot.clear(clear_session)
    gr.ChatInterface(fn=chat_with_agent, chatbot=chatbot)

/home/daniyusra/projects/manantara-playground/material-agents/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


{'messages': [HumanMessage(content='How does ratings for the romantic movies and action movies compare?', additional_kwargs={}, response_metadata={}, id='d22271e7-2607-4859-9a22-d8b7de8d0f42'), AIMessage(content='The average IMDB rating for action movies is 6.38, while there is no data available for romantic movies in the query results. This indicates that, based on the dataset, action movies have a higher average rating than romantic movies. The bar chart would show a single bar for action movies at 6.38, with no bar for romantic movies, highlighting the data gap. Without any romantic movie data, a direct comparison cannot be made. \n\n[Bar chart: One bar labeled "Action" at 6.38, no bar for "Romantic"]', additional_kwargs={}, response_metadata={'model': 'qwen3:4b-instruct-2507-q4_K_M', 'created_at': '2026-02-11T01:25:15.050947765Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4705550524, 'load_duration': 129644623, 'prompt_eval_count': 240, 'prompt_eval_duration': 12911827

Traceback (most recent call last):
  File "/home/daniyusra/projects/manantara-playground/material-agents/.venv/lib/python3.11/site-packages/gradio/queueing.py", line 766, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/daniyusra/projects/manantara-playground/material-agents/.venv/lib/python3.11/site-packages/gradio/route_utils.py", line 355, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/daniyusra/projects/manantara-playground/material-agents/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 2152, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/daniyusra/projects/manantara-playground/material-agents/.venv/lib/python3.11/site-packages/gradio/blocks.py", line 1627, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^